Do the airflow tutorial https://airflow.apache.org/tutorial.html and then set up the following:

I would like you to build a DAG (airflow term) that runs a Postgres Query (join and group by) and inserts the result into the database again. 
Setting a Postgres database can be done on your local machine (google is your friend :-) )

Hint 1: To connect airflow to the database, you should the airflow conn_id with a saved connection in your airflow web interface. 
Hint 2: You can do all this with the PostgresOperator from airflow. 

In [7]:
# The DAG object; we'll need this to instantiate a DAG
from airflow import DAG

# Operators; we need this to operate!
from airflow.operators.bash_operator import BashOperator

from datetime import datetime, timedelta

In [2]:
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': datetime(2015, 6, 1),
    'email': ['airflow@example.com'],
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
    # 'queue': 'bash_queue',
    # 'pool': 'backfill',
    # 'priority_weight': 10,
    # 'end_date': datetime(2016, 1, 1),
}

dag = DAG('tutorial', default_args=default_args, schedule_interval=timedelta(1))

In [3]:
t1 = BashOperator(
    task_id='print_date',
    bash_command='date',
    dag=dag)

t2 = BashOperator(
    task_id='sleep',
    bash_command='sleep 5',
    retries=3,
    dag=dag)

In [4]:
templated_command = """
    {% for i in range(5) %}
        echo "{{ ds }}"
        echo "{{ macros.ds_add(ds, 7) }}"
        echo "{{ params.myparam }}"
    {% endfor }
    """

t3 = BashOperator(
    task_id='templated',
    bash_command=templated_command,
    params={'my_param' : 'Parameter I passed in'},
    dag=dag)

In [5]:
t2.set_upstream(t1)
t3.set_upstream(t1)